In [ ]:
## How to work with custom models in Llama Index 

### LLMer

In [ ]:
import grpc
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata
)

import hf_course.llmer.grpc.chats_pb2 as chats_pb2
import hf_course.llmer.grpc.chats_pb2_grpc as chats_pb2_grpc 

In [ ]:
class MyCustomLLM(CustomLLM):
    def __init__(self):
        llmer_address = "localhost:50051"
        channel = grpc.insecure_channel(llmer_address)  
        client = chats_pb2_grpc.LLMChatServiceStub(channel=channel)
        session_request = chats_pb2.StartSessionRequest(
                mode="QuestionAnsweringChatBot",
                user="user123",
        )
        session_id = client.StartSession(session_request)
        session_id = session_id
                    
    """Needs to implement these three"""
    # @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=4096,  # where does this field matter
            num_output=256,       # where does this field matter
            model_name="my-test-model",
        )
    def complete(self, prompt: str, formatted: bool = False, **kwargs) -> CompletionResponse:
        # FIXME: the base class does not allow me to define fields for some reason, find a way 
        llmer_address = "localhost:50051"
        channel = grpc.insecure_channel(llmer_address)  
        client = chats_pb2_grpc.LLMChatServiceStub(channel=channel)
        session_request = chats_pb2.StartSessionRequest(
                mode="QuestionAnsweringChatBot",
                user="user123",
        )
        session_id_response = client.StartSession(session_request)
        session_id = session_id_response.id
        print(session_id)
        message_question = chats_pb2.Question(
            session_id=session_id,
            system_prompt="somethings",
            question_prompt=prompt
        )
        answer = client.SendMessage(message_question)

        return CompletionResponse(text=answer.text)

    def stream_complete(self, prompt: str, formatted: bool = False, **kwargs) -> CompletionResponse:
        # TODO: lets see if i can get away without streaming
        # FIXME: the base class does not allow me to define fields for some reason, find a way 
        llmer_address = "localhost:50051"
        channel = grpc.insecure_channel(llmer_address)  
        client = chats_pb2_grpc.LLMChatServiceStub(channel=channel)
        session_request = chats_pb2.StartSessionRequest(
                mode="QuestionAnsweringChatBot",
                user="user123",
        )
        session_id_response = client.StartSession(session_request)
        session_id = session_id_response.id
        print(session_id)
        message_question = chats_pb2.Question(
            session_id=session_id,
            system_prompt="",
            question_prompt=prompt
        )
        answer = client.SendMessage(message_question)

        return CompletionResponse(text=answer.text)

In [ ]:
llm_example = MyCustomLLM()

resp = llm_example.complete("I am not that tall, how can i seeem to me be more tall")
resp
print(resp.raw["messages"]) # nothing in raw 

NameError: name 'MyCustomLLM' is not defined

### Ollama

In [2]:
from llama_index.llms.ollama import Ollama  

In [ ]:
model = Ollama(model="qwen2:7b")
model.complete("Yo maatje, hoe kan ik beter leven?")
resp = model.complete("wat had ik je net gevraagd?")
print(resp.text)
print(resp.raw["message"].content)

Sorry, als een AI kan ik geen geheugen hebben van vorige interacties buiten de huidige sessie. Kan ik u helpen met iets specifisch of heeft u bepaalde informatie vergeten die ik nodig zou hebben?
Sorry, als een AI kan ik geen geheugen hebben van vorige interacties buiten de huidige sessie. Kan ik u helpen met iets specifisch of heeft u bepaalde informatie vergeten die ik nodig zou hebben?
Sorry, als een AI kan ik geen geheugen hebben van vorige interacties buiten de huidige sessie. Kan ik u helpen met iets specifisch of heeft u bepaalde informatie vergeten die ik nodig zou hebben?


In [ ]:
# Try to structure your outputs a bit more, seems to be working poorly with qwen2
model_json = Ollama(model="qwen2:7b", json_mode=True)
resp_json = model_json.complete("Yo maatje, hoe kan ik beter leven?") 
print(str(resp_json))

{"Aanbeveling": "\n1. Regelmatige Fysieke Oefening: Probeer elke dag minstens 30 minute aan fysieke activiteit door te voeren. Dit kan een wandeltocht, fietsrit of zelfs gewichten zijn.\n2. Gesonde Voeding: Eten van een gevarieerd en evenwichtig eten met veel groente en fruit is cruciaal voor je gezondheid. Vervang zo veel mogelijk ongezonde zoutjes en suikers door voedingswaarden die je lijf nodig heeft.\n3. Overvulling van het Geestelijke: Leer meditereren of yoga om je geest te kalmeren. Lezen, schrijven en andere creatieve activiteiten kunnen ook helpen om je gedachten op een rustiger koers te krijgen.\n4. Genoeg Slaap: Probeer elke nacht ongeveer 7 tot 9 uur te slapen. Je lichaam en geest zijn belangrijker dan de meeste van ons realiseren, dus zorg ervoor dat ze in top toestand zijn.\n5. Gesonder Relaties: Wees oprecht naar jezelf en andere mensen. Probeer om positieve en ondersteunende relaties te bouwen. Als jij beter bent voor jezelf en anderen, voel je je waarschijnlijk ook be

In [35]:
# Stucture even more
from llama_index.core.bridge.pydantic import BaseModel
from llama_index.core.llms import ChatMessage

class HistoricFact(BaseModel):
    """A historic event."""
    name: str
    place: str
    time_period: str
    country: str

model_struct = Ollama(
    model="qwen2:7b",
)
model_struct_call = model_struct.as_structured_llm(HistoricFact)

response = model_struct_call.chat([ChatMessage(role="user", content="Name a famous historic event")])
print(response.message.content)

response = model_struct_call.complete("Name a famous historic event")
print(response)

{"name":"World War II","place":"Global, spanning Europe, Africa, Asia and the Pacific regions","time_period":"1939-1945","country":"Involved countries include Germany, Japan, Italy, Britain, France, Russia, USA among others"}
{"name":"The American Revolutionary War","place":"Colonies of British America and the British Empire","time_period":"1775-1783","country":"United States of America"}


In [ ]:
response = model.stream_complete("wat eet je over Prussia")
for r in response:
    print(r.delta, end=" ")
model.complete("wat eet je over Prussia")

Het  kw amen  de  Pool se ,  D uit se  en  O ost -E u rop ese  culture n  sam env allen  in  Pr us se ,  wat  veel  vari atie  bied t  in  term en  van  vo eds el .  Hier  zijn  en ke le  klass ie ke  gere chten  die  in  Pr us se  worden  ge get en :

 1 .  Big os :  Dit  is  een  tradition eel  Pool se  gro ent eg ro ente b ou ill iet  wa aron der  k arn ar be ids ,  pap rika 's ,  kn ol r atten ,  o lij ven ,  ger uit en  var k ens v le es  en  w ort els  worden  ing ede kt .

 2 .  Pier ogi :  Deze  kleine  v le es -  of  gro ent ep ak ket jes  zijn  een  gew a arde erd  gere cht  in  veel  land en  van  O ost -E u ropa ,  wa aron der  Pol en .

 3 .  K omp ot :  Dit  is  een  tradition eel  Pool se  drank je  g emaakt  van  ged ik ke  fruit bro od jes  zoals  appel ,  appel  en /of  ab ri ko zen .

 4 .  D ö ner k ab ob :  Dit  is  een  Turk se  ke uze  die  in  veel  land en  van  Europa  te  vinden  is ,  wa aron der  Pr us se .

 5 .  K arp aska  Dad lic ia :  Een  spec ia al  

In [39]:

from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = model.chat(messages)

print(type(resp.message))
print(resp.message.blocks[0].text)

print(type(resp.raw["message"]))
print(resp.raw["message"].content)

<class 'llama_index.core.base.llms.types.ChatMessage'>
Ahoy there, matey! I be Captain彩霞掠海 (Cai Xia Luehai), the most dashing and daring captain in all the seas. Call me Captain Sunsetsails or just plain old Cai for short. What brings you to my deck today?
<class 'ollama._types.Message'>
Ahoy there, matey! I be Captain彩霞掠海 (Cai Xia Luehai), the most dashing and daring captain in all the seas. Call me Captain Sunsetsails or just plain old Cai for short. What brings you to my deck today?


## Ollama Vision 


In [ ]:
import ollama

res = ollama.chat(
	model="llava",
	messages=[
		{
			'role': 'user',
			'content': 'Describe this image:',
			'images': ['/Users/maksim.rostov/Desktop/test5.png']
		}
	]
)

print(res['message']['content'])

 The image shows a sign with the text "Ollam Vision". The background of the sign is white, and the text is in black. The sign includes a URL or website address at the bottom. Below the main sign, there's another smaller banner with the text "Welcome to the era of open-source multimodal models." The image is taken from an angle where the top of the sign is visible but not fully captured in the frame. The overall style of the image is promotional or informative, suggesting a connection to the tech industry or software development. 


In [42]:
from llama_index.core.llms import ChatMessage, TextBlock, ImageBlock
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="llava",
)

messages = [
    ChatMessage(
        role="user",
        blocks=[
            TextBlock(text="What does the image say"),
            ImageBlock(path="/Users/maksim.rostov/Desktop/test5.png"),
        ],
    ),
]

resp = llm.chat(messages)
print(resp)

assistant:  The image shows a sign or banner that says "Ollam Vision," which appears to be an invitation or advertisement for some kind of event, gathering, or organization related to open-source multimodal models. It is welcoming visitors and is located in an area called "The Era of Open Source." 
